<a href="https://colab.research.google.com/github/pramod1997/Path-To-Data-Science/blob/master/image_classifier_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# improvement from prev version  -> batch norm  
# 4st attempt on image classification  cifar10 
# 
# % test accuracy :-< 81 percent , still overfitting


In [1]:
import pandas as pd
import numpy as np
import keras 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import cv2 as cv2

Using TensorFlow backend.


In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense ,Input , Flatten ,Conv2D ,Activation  , MaxPooling2D , Dropout, BatchNormalization
from keras.models import Sequential

In [0]:
from keras.datasets import cifar10

In [0]:
data_augementation = False  # we will use this later

In [5]:
((train_image , train_labels) , (test_image , test_labels)) = cifar10.load_data()

170500096/170498071 [==============================] - 34s 0us/step


In [0]:
# Convert class vectors to binary class matrices.
num_classes =10 # 10  labels are present 
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

In [7]:
test_labels  ### one hot encoding

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [8]:
# eda 
train_image.shape , test_image.shape  , train_labels , test_labels

((50000, 32, 32, 3), (10000, 32, 32, 3), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]], dtype=float32))

In [0]:
train_image = train_image.astype('float32')
test_image = test_image.astype('float32')
train_image/=255
test_image/=255

In [10]:
### model 
model = Sequential()

model.add(Conv2D( filters = 32 , kernel_size = (3,3), padding='same' , input_shape=(32,32,3)))
model.add(Activation('relu'))


model.add(Conv2D( filters = 32 , kernel_size = (3,3), padding='same' ))
model.add(Activation('relu'))


model.add(MaxPooling2D(pool_size =(2,2)))
model.add(BatchNormalization())   ###  add after pooling layer 
model.add(Dropout(0.25))


model.add(Conv2D( filters = 64 , kernel_size = (3,3), padding='same'))
model.add(Activation('relu'))


model.add(Conv2D( filters = 64 , kernel_size = (3,3), padding='same'))
model.add(Activation('relu'))


model.add(MaxPooling2D(pool_size =(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))





model.add(Flatten())

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(optimizer = 'adam' , 
             loss= 'categorical_crossentropy',
             metrics = ['accuracy'])


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
__________

In [13]:
train_image.shape , train_labels.shape  ## label shape changed for multiclass classification 

((50000, 32, 32, 3), (50000, 10))

In [0]:
# real time augementation on the data 
datagenerator = ImageDataGenerator()
datagenerator.fit(train_image)
#(x_train , y_train) = datagenerator.flow(train_image,train_labels, batch_size =32)





In [15]:
#model.fit(train_image , train_labels, epochs = 100)  
model.fit_generator(datagenerator.flow(train_image,train_labels, batch_size =32) ,
                    samples_per_epoch = len(train_image),epochs =100)   ## fit genrator because the images cant fit in memory

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=100, steps_per_epoch=1562)`
  


Epoch 1/100
1562/1562 [==============================] - 36s 23ms/step - loss: 1.5104 - acc: 0.4695
Epoch 2/100
1562/1562 [==============================] - 31s 20ms/step - loss: 1.1004 - acc: 0.6149
Epoch 3/100
1562/1562 [==============================] - 31s 20ms/step - loss: 0.9328 - acc: 0.6715
Epoch 4/100
1562/1562 [==============================] - 31s 20ms/step - loss: 0.8306 - acc: 0.7104
Epoch 5/100
1562/1562 [==============================] - 31s 20ms/step - loss: 0.7447 - acc: 0.7387
Epoch 6/100
1562/1562 [==============================] - 32s 20ms/step - loss: 0.6883 - acc: 0.7593
Epoch 7/100
1562/1562 [==============================] - 31s 20ms/step - loss: 0.6265 - acc: 0.7805
Epoch 8/100
1562/1562 [==============================] - 31s 20ms/step - loss: 0.5799 - acc: 0.7991
Epoch 9/100
1562/1562 [==============================] - 30s 19ms/step - loss: 0.5403 - acc: 0.8122
Epoch 10/100
1562/1562 [==============================] - 30s 19ms/step - loss: 0.5062 - acc: 0.8231

In [16]:
test_loss , test_acc = model.evaluate(test_image , test_labels)

10000/10000 [==============================] - 2s 155us/step


In [17]:
test_loss , test_acc



(0.7873610929131508, 0.8196)